
Thư viện



In [139]:
pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [140]:
import numpy as np
import pandas as pd
import scipy.io
import io
from sklearn.model_selection import KFold
from sklearn import svm
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, r2_score, mean_squared_error, roc_auc_score, f1_score
from scipy.sparse import *

from catboost import CatBoostRegressor, CatBoostClassifier
import math
import time

In [141]:
def construct_W(X, **kwargs):
  

    # default metric is 'cosine'
    if 'metric' not in kwargs.keys():
        kwargs['metric'] = 'cosine'

    # default neighbor mode is 'knn' and default neighbor size is 5
    if 'neighbor_mode' not in kwargs.keys():
        kwargs['neighbor_mode'] = 'knn'
    if kwargs['neighbor_mode'] == 'knn' and 'k' not in kwargs.keys():
        kwargs['k'] = 5
    if kwargs['neighbor_mode'] == 'supervised' and 'k' not in kwargs.keys():
        kwargs['k'] = 5
    if kwargs['neighbor_mode'] == 'supervised' and 'y' not in kwargs.keys():
        print ('Warning: label is required in the supervised neighborMode!!!')
        exit(0)

    # default weight mode is 'binary', default t in heat kernel mode is 1
    if 'weight_mode' not in kwargs.keys():
        kwargs['weight_mode'] = 'binary'
    if kwargs['weight_mode'] == 'heat_kernel':
        if kwargs['metric'] != 'euclidean':
            kwargs['metric'] = 'euclidean'
        if 't' not in kwargs.keys():
            kwargs['t'] = 1
    elif kwargs['weight_mode'] == 'cosine':
        if kwargs['metric'] != 'cosine':
            kwargs['metric'] = 'cosine'

    # default fisher_score and reliefF mode are 'false'
    if 'fisher_score' not in kwargs.keys():
        kwargs['fisher_score'] = False
    if 'reliefF' not in kwargs.keys():
        kwargs['reliefF'] = False

    n_samples, n_features = np.shape(X)

    # choose 'knn' neighbor mode
    if kwargs['neighbor_mode'] == 'knn':
        k = kwargs['k']
        if kwargs['weight_mode'] == 'binary':
            if kwargs['metric'] == 'euclidean':
                # compute pairwise euclidean distances
                D = pairwise_distances(X)
                D **= 2
                # sort the distance matrix D in ascending order
                dump = np.sort(D, axis=1)
                idx = np.argsort(D, axis=1)
                # choose the k-nearest neighbors for each instance
                idx_new = idx[:, 0:k+1]
                G = np.zeros((n_samples*(k+1), 3))
                G[:, 0] = np.tile(np.arange(n_samples), (k+1, 1)).reshape(-1)
                G[:, 1] = np.ravel(idx_new, order='F')
                G[:, 2] = 1
                # build the sparse affinity matrix W
                W = csc_matrix((G[:, 2], (G[:, 0], G[:, 1])), shape=(n_samples, n_samples))
                bigger = np.transpose(W) > W
                W = W - W.multiply(bigger) + np.transpose(W).multiply(bigger)
                return W

            elif kwargs['metric'] == 'cosine':
                # normalize the data first
                X_normalized = np.power(np.sum(X*X, axis=1), 0.5)
                for i in range(n_samples):
                    X[i, :] = X[i, :]/max(1e-12, X_normalized[i])
                # compute pairwise cosine distances
                D_cosine = np.dot(X, np.transpose(X))
                # sort the distance matrix D in descending order
                dump = np.sort(-D_cosine, axis=1)
                idx = np.argsort(-D_cosine, axis=1)
                idx_new = idx[:, 0:k+1]
                G = np.zeros((n_samples*(k+1), 3))
                G[:, 0] = np.tile(np.arange(n_samples), (k+1, 1)).reshape(-1)
                G[:, 1] = np.ravel(idx_new, order='F')
                G[:, 2] = 1
                # build the sparse affinity matrix W
                W = csc_matrix((G[:, 2], (G[:, 0], G[:, 1])), shape=(n_samples, n_samples))
                bigger = np.transpose(W) > W
                W = W - W.multiply(bigger) + np.transpose(W).multiply(bigger)
                return W

        elif kwargs['weight_mode'] == 'heat_kernel':
            t = kwargs['t']
            # compute pairwise euclidean distances
            D = pairwise_distances(X)
            D **= 2
            # sort the distance matrix D in ascending order
            dump = np.sort(D, axis=1)
            idx = np.argsort(D, axis=1)
            idx_new = idx[:, 0:k+1]
            dump_new = dump[:, 0:k+1]
            # compute the pairwise heat kernel distances
            dump_heat_kernel = np.exp(-dump_new/(2*t*t))
            G = np.zeros((n_samples*(k+1), 3))
            G[:, 0] = np.tile(np.arange(n_samples), (k+1, 1)).reshape(-1)
            G[:, 1] = np.ravel(idx_new, order='F')
            G[:, 2] = np.ravel(dump_heat_kernel, order='F')
            # build the sparse affinity matrix W
            W = csc_matrix((G[:, 2], (G[:, 0], G[:, 1])), shape=(n_samples, n_samples))
            bigger = np.transpose(W) > W
            W = W - W.multiply(bigger) + np.transpose(W).multiply(bigger)
            return W

        elif kwargs['weight_mode'] == 'cosine':
            # normalize the data first
            X_normalized = np.power(np.sum(X*X, axis=1), 0.5)
            for i in range(n_samples):
                    X[i, :] = X[i, :]/max(1e-12, X_normalized[i])
            # compute pairwise cosine distances
            D_cosine = np.dot(X, np.transpose(X))
            # sort the distance matrix D in ascending order
            dump = np.sort(-D_cosine, axis=1)
            idx = np.argsort(-D_cosine, axis=1)
            idx_new = idx[:, 0:k+1]
            dump_new = -dump[:, 0:k+1]
            G = np.zeros((n_samples*(k+1), 3))
            G[:, 0] = np.tile(np.arange(n_samples), (k+1, 1)).reshape(-1)
            G[:, 1] = np.ravel(idx_new, order='F')
            G[:, 2] = np.ravel(dump_new, order='F')
            # build the sparse affinity matrix W
            W = csc_matrix((G[:, 2], (G[:, 0], G[:, 1])), shape=(n_samples, n_samples))
            bigger = np.transpose(W) > W
            W = W - W.multiply(bigger) + np.transpose(W).multiply(bigger)
            return W

    # choose supervised neighborMode
    elif kwargs['neighbor_mode'] == 'supervised':
        k = kwargs['k']
        # get true labels and the number of classes
        y = kwargs['y']
        label = np.unique(y)
        n_classes = np.unique(y).size
        # construct the weight matrix W in a fisherScore way, W_ij = 1/n_l if yi = yj = l, otherwise W_ij = 0
        if kwargs['fisher_score'] is True:
            W = lil_matrix((n_samples, n_samples))
            for i in range(n_classes):
                class_idx = (y == label[i])
                class_idx_all = (class_idx[:, np.newaxis] & class_idx[np.newaxis, :])
                W[class_idx_all] = 1.0/np.sum(np.sum(class_idx))
            return W

        # construct the weight matrix W in a reliefF way, NH(x) and NM(x,y) denotes a set of k nearest
        # points to x with the same class as x, a different class (the class y), respectively. W_ij = 1 if i = j;
        # W_ij = 1/k if x_j \in NH(x_i); W_ij = -1/(c-1)k if x_j \in NM(x_i, y)
        if kwargs['reliefF'] is True:
            # when xj in NH(xi)
            G = np.zeros((n_samples*(k+1), 3))
            id_now = 0
            for i in range(n_classes):
                class_idx = np.column_stack(np.where(y == label[i]))[:, 0]
                D = pairwise_distances(X[class_idx, :])
                D **= 2
                idx = np.argsort(D, axis=1)
                idx_new = idx[:, 0:k+1]
                n_smp_class = (class_idx[idx_new[:]]).size
                if len(class_idx) <= k:
                    k = len(class_idx) - 1
                G[id_now:n_smp_class+id_now, 0] = np.tile(class_idx, (k+1, 1)).reshape(-1)
                G[id_now:n_smp_class+id_now, 1] = np.ravel(class_idx[idx_new[:]], order='F')
                G[id_now:n_smp_class+id_now, 2] = 1.0/k
                id_now += n_smp_class
            W1 = csc_matrix((G[:, 2], (G[:, 0], G[:, 1])), shape=(n_samples, n_samples))
            # when i = j, W_ij = 1
            for i in range(n_samples):
                W1[i, i] = 1
            # when x_j in NM(x_i, y)
            G = np.zeros((n_samples*k*(n_classes - 1), 3))
            id_now = 0
            for i in range(n_classes):
                class_idx1 = np.column_stack(np.where(y == label[i]))[:, 0]
                X1 = X[class_idx1, :]
                for j in range(n_classes):
                    if label[j] != label[i]:
                        class_idx2 = np.column_stack(np.where(y == label[j]))[:, 0]
                        X2 = X[class_idx2, :]
                        D = pairwise_distances(X1, X2)
                        idx = np.argsort(D, axis=1)
                        idx_new = idx[:, 0:k]
                        n_smp_class = len(class_idx1)*k
                        G[id_now:n_smp_class+id_now, 0] = np.tile(class_idx1, (k, 1)).reshape(-1)
                        G[id_now:n_smp_class+id_now, 1] = np.ravel(class_idx2[idx_new[:]], order='F')
                        G[id_now:n_smp_class+id_now, 2] = -1.0/((n_classes-1)*k)
                        id_now += n_smp_class
            W2 = csc_matrix((G[:, 2], (G[:, 0], G[:, 1])), shape=(n_samples, n_samples))
            bigger = np.transpose(W2) > W2
            W2 = W2 - W2.multiply(bigger) + np.transpose(W2).multiply(bigger)
            W = W1 + W2
            return W

        if kwargs['weight_mode'] == 'binary':
            if kwargs['metric'] == 'euclidean':
                G = np.zeros((n_samples*(k+1), 3))
                id_now = 0
                for i in range(n_classes):
                    class_idx = np.column_stack(np.where(y == label[i]))[:, 0]
                    # compute pairwise euclidean distances for instances in class i
                    D = pairwise_distances(X[class_idx, :])
                    D **= 2
                    # sort the distance matrix D in ascending order for instances in class i
                    idx = np.argsort(D, axis=1)
                    idx_new = idx[:, 0:k+1]
                    n_smp_class = len(class_idx)*(k+1)
                    G[id_now:n_smp_class+id_now, 0] = np.tile(class_idx, (k+1, 1)).reshape(-1)
                    G[id_now:n_smp_class+id_now, 1] = np.ravel(class_idx[idx_new[:]], order='F')
                    G[id_now:n_smp_class+id_now, 2] = 1
                    id_now += n_smp_class
                # build the sparse affinity matrix W
                W = csc_matrix((G[:, 2], (G[:, 0], G[:, 1])), shape=(n_samples, n_samples))
                bigger = np.transpose(W) > W
                W = W - W.multiply(bigger) + np.transpose(W).multiply(bigger)
                return W

            if kwargs['metric'] == 'cosine':
                # normalize the data first
                X_normalized = np.power(np.sum(X*X, axis=1), 0.5)
                for i in range(n_samples):
                    X[i, :] = X[i, :]/max(1e-12, X_normalized[i])
                G = np.zeros((n_samples*(k+1), 3))
                id_now = 0
                for i in range(n_classes):
                    class_idx = np.column_stack(np.where(y == label[i]))[:, 0]
                    # compute pairwise cosine distances for instances in class i
                    D_cosine = np.dot(X[class_idx, :], np.transpose(X[class_idx, :]))
                    # sort the distance matrix D in descending order for instances in class i
                    idx = np.argsort(-D_cosine, axis=1)
                    idx_new = idx[:, 0:k+1]
                    n_smp_class = len(class_idx)*(k+1)
                    G[id_now:n_smp_class+id_now, 0] = np.tile(class_idx, (k+1, 1)).reshape(-1)
                    G[id_now:n_smp_class+id_now, 1] = np.ravel(class_idx[idx_new[:]], order='F')
                    G[id_now:n_smp_class+id_now, 2] = 1
                    id_now += n_smp_class
                # build the sparse affinity matrix W
                W = csc_matrix((G[:, 2], (G[:, 0], G[:, 1])), shape=(n_samples, n_samples))
                bigger = np.transpose(W) > W
                W = W - W.multiply(bigger) + np.transpose(W).multiply(bigger)
                return W

        elif kwargs['weight_mode'] == 'heat_kernel':
            G = np.zeros((n_samples*(k+1), 3))
            id_now = 0
            for i in range(n_classes):
                class_idx = np.column_stack(np.where(y == label[i]))[:, 0]
                # compute pairwise cosine distances for instances in class i
                D = pairwise_distances(X[class_idx, :])
                D **= 2
                # sort the distance matrix D in ascending order for instances in class i
                dump = np.sort(D, axis=1)
                idx = np.argsort(D, axis=1)
                idx_new = idx[:, 0:k+1]
                dump_new = dump[:, 0:k+1]
                t = kwargs['t']
                # compute pairwise heat kernel distances for instances in class i
                dump_heat_kernel = np.exp(-dump_new/(2*t*t))
                n_smp_class = len(class_idx)*(k+1)
                G[id_now:n_smp_class+id_now, 0] = np.tile(class_idx, (k+1, 1)).reshape(-1)
                G[id_now:n_smp_class+id_now, 1] = np.ravel(class_idx[idx_new[:]], order='F')
                G[id_now:n_smp_class+id_now, 2] = np.ravel(dump_heat_kernel, order='F')
                id_now += n_smp_class
            # build the sparse affinity matrix W
            W = csc_matrix((G[:, 2], (G[:, 0], G[:, 1])), shape=(n_samples, n_samples))
            bigger = np.transpose(W) > W
            W = W - W.multiply(bigger) + np.transpose(W).multiply(bigger)
            return W

        elif kwargs['weight_mode'] == 'cosine':
            # normalize the data first
            X_normalized = np.power(np.sum(X*X, axis=1), 0.5)
            for i in range(n_samples):
                X[i, :] = X[i, :]/max(1e-12, X_normalized[i])
            G = np.zeros((n_samples*(k+1), 3))
            id_now = 0
            for i in range(n_classes):
                class_idx = np.column_stack(np.where(y == label[i]))[:, 0]
                # compute pairwise cosine distances for instances in class i
                D_cosine = np.dot(X[class_idx, :], np.transpose(X[class_idx, :]))
                # sort the distance matrix D in descending order for instances in class i
                dump = np.sort(-D_cosine, axis=1)
                idx = np.argsort(-D_cosine, axis=1)
                idx_new = idx[:, 0:k+1]
                dump_new = -dump[:, 0:k+1]
                n_smp_class = len(class_idx)*(k+1)
                G[id_now:n_smp_class+id_now, 0] = np.tile(class_idx, (k+1, 1)).reshape(-1)
                G[id_now:n_smp_class+id_now, 1] = np.ravel(class_idx[idx_new[:]], order='F')
                G[id_now:n_smp_class+id_now, 2] = np.ravel(dump_new, order='F')
                id_now += n_smp_class
            # build the sparse affinity matrix W
            W = csc_matrix((G[:, 2], (G[:, 0], G[:, 1])), shape=(n_samples, n_samples))
            bigger = np.transpose(W) > W
            W = W - W.multiply(bigger) + np.transpose(W).multiply(bigger)
            return W

In [142]:
def fisher_score(X, y):

    # Construct weight matrix W in a fisherScore way
    kwargs = {"neighbor_mode": "supervised", "fisher_score": True, 'y': y}
    W = construct_W(X, **kwargs)

    # build the diagonal D matrix from affinity matrix W
    D = np.array(W.sum(axis=1))
    #L = W
    tmp = np.dot(np.transpose(D), X)
    D = diags(np.transpose(D), [0])
    #Xt = np.transpose(X)
    #t1 = np.transpose(np.dot(np.transpose(X), D.todense()))
    #t2 = np.transpose(np.dot(np.transpose(X), W.todense()))
    # compute the numerator of Lr
    D_prime = np.sum(np.multiply(np.transpose(np.dot(np.transpose(X), D.todense())), X), 0) - np.multiply(tmp, tmp)/D.sum()
    # compute the denominator of Lr
    L_prime = np.sum(np.multiply(np.transpose(np.dot(np.transpose(X), W.todense())), X), 0) - np.multiply(tmp, tmp)/D.sum()
    # avoid the denominator of Lr to be 0
    D_prime[D_prime < 1e-12] = 10000
    lap_score = 1 - np.array(np.multiply(L_prime, 1/D_prime))[0, :]

    # compute fisher score from laplacian score, where fisher_score = 1/lap_score - 1
    score = 1.0/lap_score - 1
    return np.transpose(score)


def feature_ranking(score):
    """
    Rank features in descending order according to fisher score, the larger the fisher score, the more important the
    feature is
    """
    idx = np.argsort(score)
    return idx[::-1]

In [143]:
from google.colab import drive 
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [144]:
# load data

dataset = 'CreditRisk.csv'
link = '/content/drive/MyDrive/FeatureSelection/dataset/' + dataset
df=pd.read_csv(link)
nRow, nCol = df.shape
print(f'There are {nRow} rows and {nCol} columns')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (17,45,53) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


There are 855969 rows and 73 columns


In [145]:
df.columns
(df['y'].value_counts()) / len(df) * 100

0    94.571416
1     5.428584
Name: y, dtype: float64

In [146]:
n_samples = 11000;
df = df.groupby('y', group_keys=False).apply(lambda x: x.sample(frac=n_samples/nRow))
df.shape
(df['y'].value_counts()) / len(df) * 100

0    94.572727
1     5.427273
Name: y, dtype: float64

In [147]:
#df = df.dropna(axis='rows')

cat_columns = df.select_dtypes(['object']).columns
if not cat_columns.empty:
  df[cat_columns] = df[cat_columns].apply(lambda x: pd.factorize(x)[0])
df = df.fillna(0)
df.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,y
362695,18494778,20657467,10000,10000,10000.0,0,8.39,315.17,0,0,...,0.0,0.0,0.0,0.0,0.0,12400.0,0.0,0.0,0.0,0
107325,6301559,7832773,13375,13375,13375.0,0,17.56,480.60,1,1,...,0.0,0.0,0.0,0.0,0.0,32300.0,0.0,0.0,0.0,0
503664,65472734,70059466,12000,12000,12000.0,1,12.05,267.24,2,2,...,0.0,0.0,0.0,0.0,0.0,10500.0,0.0,0.0,0.0,0
730819,48050323,51290041,13600,13600,13600.0,0,19.52,502.11,3,3,...,0.0,0.0,0.0,0.0,0.0,26600.0,0.0,0.0,0.0,0
559179,61401731,65520535,25000,25000,25000.0,0,12.29,833.83,2,2,...,0.0,0.0,0.0,0.0,0.0,67300.0,0.0,0.0,0.0,0


In [148]:
X = df.drop(columns=['id','y']).to_numpy()

features_label = df.columns.difference(['y']).to_numpy()
y = df['y'].to_numpy()
n_samples, n_features = X.shape
X.shape
del df

In [149]:
start_time=time.time()

# obtain the score of each feature on the training set. The larger the feature score is, the more important the feature is
score = fisher_score(X, y)

# rank features in descending order according to score
idx = feature_ranking(score)

#print(score)
print("Ranking: ")
print(idx)
#print(features_label[idx[0:num_fea]])


Ranking: 
[40 41 44 16  9 42 18 45 34 33  0 14  5 39 32 37 43 24 67 38 55 30  4  8
 17 12 36 35  7 47 66 62 65 25 11 64 59 57 54 27 56 29 63 61 60 58 31 68
 28  6 69 19 70 20 23  1 10 26  2 53 49 50 51 52 21  3 46 13 22 15 48]


In [150]:
idx_top10 = idx[0:int(n_features/10)]
print("--------------------------------------")
print("index of top  10 : %s" %idx_top10)
print("Top 10 feature: %s " %features_label[idx_top10])

--------------------------------------
index of top  10 : [40 41 44 16  9 42 18]
Top 10 feature: ['open_acc_6m' 'open_il_12m' 'open_rv_12m' 'funded_amnt_inv' 'desc'
 'open_il_24m' 'home_ownership'] 


In [151]:
idx_top15 = idx[0:int(15*n_features/100)]
print("--------------------------------------")
print("index of top  15 : %s" %idx_top15)
print("Top 15 feature: %s " %features_label[idx_top15])

--------------------------------------
index of top  15 : [40 41 44 16  9 42 18 45 34 33]
Top 15 feature: ['open_acc_6m' 'open_il_12m' 'open_rv_12m' 'funded_amnt_inv' 'desc'
 'open_il_24m' 'home_ownership' 'open_rv_24m' 'mths_since_last_delinq'
 'member_id'] 


In [152]:
idx_top25 = idx[0:int(n_features/4)]
print("--------------------------------------")
print("index of top  25: %s" %idx_top25)
print("Top 25 feature: %s " %features_label[idx_top25])

print("--- %s seconds ---" % (time.time() - start_time))

--------------------------------------
index of top  25: [40 41 44 16  9 42 18 45 34 33  0 14  5 39 32 37 43]
Top 25 feature: ['open_acc_6m' 'open_il_12m' 'open_rv_12m' 'funded_amnt_inv' 'desc'
 'open_il_24m' 'home_ownership' 'open_rv_24m' 'mths_since_last_delinq'
 'member_id' 'acc_now_delinq' 'emp_title' 'application_type' 'open_acc'
 'max_bal_bc' 'mths_since_rcnt_il' 'open_il_6m'] 
--- 78.81181359291077 seconds ---


In [153]:
X_selected = X[:, idx_top10]
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=5)


In [154]:
# fit and predict

model = CatBoostClassifier(loss_function='Logloss', eval_metric='Accuracy')
model.fit(X_train, y_train)
y_predict = model.predict(X_test)

y_predict
acc = "{:.2f}".format(accuracy_score(y_test,y_predict))
auc = "{:.2f}".format(roc_auc_score(y_test, y_predict))
f1 = "{:.2f}".format(f1_score(y_test,y_predict))
print('Accuracy =', accuracy_score(y_test,y_predict))
print('ROC AUC =', roc_auc_score(y_test, y_predict ))
print('F1 =', f1_score(y_test,y_predict))

Learning rate set to 0.026075
0:	learn: 0.9850000	total: 5.3ms	remaining: 5.29s
1:	learn: 0.9889773	total: 10.4ms	remaining: 5.2s
2:	learn: 0.9889773	total: 15.2ms	remaining: 5.04s
3:	learn: 0.9888636	total: 20ms	remaining: 4.98s
4:	learn: 0.9888636	total: 24.8ms	remaining: 4.94s
5:	learn: 0.9889773	total: 29.8ms	remaining: 4.93s
6:	learn: 0.9892045	total: 34.7ms	remaining: 4.92s
7:	learn: 0.9890909	total: 39.3ms	remaining: 4.88s
8:	learn: 0.9890909	total: 43.9ms	remaining: 4.84s
9:	learn: 0.9890909	total: 48.5ms	remaining: 4.81s
10:	learn: 0.9888636	total: 54.5ms	remaining: 4.9s
11:	learn: 0.9890909	total: 61.4ms	remaining: 5.06s
12:	learn: 0.9896591	total: 66.1ms	remaining: 5.02s
13:	learn: 0.9888636	total: 72.1ms	remaining: 5.08s
14:	learn: 0.9888636	total: 77.6ms	remaining: 5.1s
15:	learn: 0.9895455	total: 82.4ms	remaining: 5.07s
16:	learn: 0.9889773	total: 87ms	remaining: 5.03s
17:	learn: 0.9896591	total: 95.3ms	remaining: 5.2s
18:	learn: 0.9896591	total: 99.9ms	remaining: 5.16s
1

In [155]:
df=pd.read_csv('/content/drive/MyDrive/FeatureSelection/dataset/result.csv')
df2 = pd.DataFrame({'score':'Fisher','name':[dataset], 'samples': [n_samples], 'top': '10','acc':[acc],'auc':[auc],'f1':[f1]})
df = pd.concat([df,df2], ignore_index = True)
df.to_csv('/content/drive/MyDrive/FeatureSelection/dataset/result.csv', index=False)


In [156]:
X_selected = X[:, idx_top15]
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=5)


In [157]:
# fit and predict

model = CatBoostClassifier(loss_function='Logloss', eval_metric='Accuracy')
model.fit(X_train, y_train)
y_predict = model.predict(X_test)

acc = "{:.2f}".format(accuracy_score(y_test,y_predict))
auc = "{:.2f}".format(roc_auc_score(y_test, y_predict))
f1 = "{:.2f}".format(f1_score(y_test,y_predict))
print('Accuracy =', acc)
print('ROC AUC =', auc)
print('F1 =', f1)

Learning rate set to 0.026075
0:	learn: 0.9847727	total: 7.06ms	remaining: 7.05s
1:	learn: 0.9886364	total: 13.2ms	remaining: 6.57s
2:	learn: 0.9896591	total: 19ms	remaining: 6.33s
3:	learn: 0.9895455	total: 24.7ms	remaining: 6.14s
4:	learn: 0.9895455	total: 32.3ms	remaining: 6.43s
5:	learn: 0.9887500	total: 38.2ms	remaining: 6.33s
6:	learn: 0.9895455	total: 46.6ms	remaining: 6.61s
7:	learn: 0.9895455	total: 52.5ms	remaining: 6.51s
8:	learn: 0.9896591	total: 57.8ms	remaining: 6.36s
9:	learn: 0.9892045	total: 63.3ms	remaining: 6.27s
10:	learn: 0.9886364	total: 68.9ms	remaining: 6.19s
11:	learn: 0.9887500	total: 74.4ms	remaining: 6.13s
12:	learn: 0.9888636	total: 84.7ms	remaining: 6.43s
13:	learn: 0.9889773	total: 90.3ms	remaining: 6.36s
14:	learn: 0.9890909	total: 96.5ms	remaining: 6.34s
15:	learn: 0.9892045	total: 102ms	remaining: 6.29s
16:	learn: 0.9892045	total: 108ms	remaining: 6.23s
17:	learn: 0.9897727	total: 113ms	remaining: 6.18s
18:	learn: 0.9898864	total: 120ms	remaining: 6.18

In [158]:
df=pd.read_csv('/content/drive/MyDrive/FeatureSelection/dataset/result.csv')
df2 = pd.DataFrame({'score':'Fisher','name':[dataset], 'samples': [n_samples], 'top': '15','acc':[acc],'auc':[auc],'f1':[f1]})
df = pd.concat([df,df2], ignore_index = True)
df.to_csv('/content/drive/MyDrive/FeatureSelection/dataset/result.csv', index=False)


In [159]:
X_selected = X[:, idx_top25]
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=5)


In [160]:
# fit and predict

model = CatBoostClassifier(loss_function='Logloss', eval_metric='Accuracy')
model.fit(X_train, y_train)
y_predict = model.predict(X_test)

y_predict
acc = "{:.2f}".format(accuracy_score(y_test,y_predict))
auc = "{:.2f}".format(roc_auc_score(y_test, y_predict))
f1 = "{:.2f}".format(f1_score(y_test,y_predict))
print('Accuracy =', accuracy_score(y_test,y_predict))
print('ROC AUC =', roc_auc_score(y_test, y_predict ))
print('F1 =', f1_score(y_test,y_predict))

Learning rate set to 0.026075
0:	learn: 0.9839773	total: 8.24ms	remaining: 8.23s
1:	learn: 0.9869318	total: 17.8ms	remaining: 8.89s
2:	learn: 0.9875000	total: 26.6ms	remaining: 8.84s
3:	learn: 0.9876136	total: 33.4ms	remaining: 8.31s
4:	learn: 0.9889773	total: 39.9ms	remaining: 7.95s
5:	learn: 0.9893182	total: 46.2ms	remaining: 7.66s
6:	learn: 0.9894318	total: 52.9ms	remaining: 7.51s
7:	learn: 0.9895455	total: 59.7ms	remaining: 7.4s
8:	learn: 0.9894318	total: 66.4ms	remaining: 7.31s
9:	learn: 0.9890909	total: 73.2ms	remaining: 7.25s
10:	learn: 0.9893182	total: 80ms	remaining: 7.19s
11:	learn: 0.9895455	total: 86.7ms	remaining: 7.14s
12:	learn: 0.9894318	total: 93.8ms	remaining: 7.12s
13:	learn: 0.9894318	total: 101ms	remaining: 7.09s
14:	learn: 0.9896591	total: 107ms	remaining: 7.04s
15:	learn: 0.9897727	total: 114ms	remaining: 7.02s
16:	learn: 0.9897727	total: 121ms	remaining: 7.01s
17:	learn: 0.9897727	total: 129ms	remaining: 7.01s
18:	learn: 0.9897727	total: 136ms	remaining: 7s
19:	

In [161]:
df=pd.read_csv('/content/drive/MyDrive/FeatureSelection/dataset/result.csv')
df2 = pd.DataFrame({'score':'Fisher','name':[dataset], 'samples': [n_samples], 'top': '25','acc':[acc],'auc':[auc],'f1':[f1]})
df = pd.concat([df,df2], ignore_index = True)
df.to_csv('/content/drive/MyDrive/FeatureSelection/dataset/result.csv', index=False)
